In [1]:
import os
import json 

# Load the environment configuration JSON data
json_path = 'env_config.json'
with open(json_path, 'r') as file:
    env_config = json.load(file)

hf_home = env_config['HF_HOME']
# Set the HF_HOME environment variable
os.environ['HF_HOME'] = hf_home
# Set the access token to huggingface hub
access_token = env_config['access_token']
os.environ['HUGGINGFACE_HUB_TOKEN'] = access_token

# Load Model and Tokenizer

In [2]:
import transformers 
print(transformers.__version__)

from transformers import pipeline
import torch

from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import LlamaTokenizerFast


accelerator = Accelerator()
device = accelerator.device

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B"  # non-instruct version

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=access_token,
)

tokenizer = LlamaTokenizerFast.from_pretrained(model_id, token=access_token)

/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.41.0


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.84s/it]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load data and define dataloaders

In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader

imdb = load_dataset("imdb")
train_ds = imdb['train']


def collate_fn(examples):
    max_len = 512
    texts = [example['text'] for example in examples]
    texts = [text if len(text) <= max_len else text[:max_len] for text in texts]
    labels = [example['label'] for example in examples]
    messages_lambda = lambda texts: [
        {"role": "system", "content": "You are a chatbot for sentimate analysis. You can help users with their questions via concise responses of POSITIVE, or NEGATIVE."},
        # {"role": "system", "content": "You are a chatbot for sentimate analysis."},
        {"role": "user", "content": texts},
    ]
    messages = list(map(messages_lambda, texts))

    messages_with_template_applied = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    batch = tokenizer(
                messages_with_template_applied,
                add_special_tokens=False,
                padding=True,
                return_tensors="pt",
                )
    
    # find the template boundaries
    text_lens = [len(tokenizer.encode(text)) for text in texts]
    text_lens_tensor = torch.tensor(text_lens, dtype=torch.long)
    

    def apply_mask(mask_tensor, text_lens_tensor):
        batch_size, seq_len = mask_tensor.shape
        for i in range(batch_size):
            text_len = text_lens_tensor[i].item()
            mask_tensor[i, -text_len-5:-5] = 0
        return 1- mask_tensor

    mask_tensor = apply_mask(torch.ones_like(batch['input_ids']), text_lens_tensor)

    batch['context_mask'] = mask_tensor

    
    return batch


# Define batch size here!
batch_size = 16
train_dataloader = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)


# Define maskgen model

In [4]:
import torch.nn as nn 
# from models import MLP
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_blocks=5, bottleneck_dim=64):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.layers = nn.ModuleList()
        for _ in range(num_blocks):
            shortcut_layers = []
            shortcut_layers.append(nn.Linear(hidden_dim, bottleneck_dim))
            shortcut_layers.append(nn.Dropout())
            shortcut_layers.append(nn.ReLU())  # Using ReLU for simplicity; you can choose other activations as needed
            shortcut_layers.append(nn.Linear(bottleneck_dim, bottleneck_dim))
            shortcut_layers.append(nn.Dropout())
            shortcut_layers.append(nn.ReLU())
            shortcut_layers.append(nn.Linear(bottleneck_dim, hidden_dim))
            shortcut_layers.append(nn.Dropout())
            self.layers.append(nn.Sequential(*shortcut_layers))

        self.output_layer= nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.input_layer(x)
        for layer in self.layers:
            x = x + layer(x) # shortcut
        x = F.normalize(x, p=2, dim=-1)
        return self.output_layer(x)

def similarity_measure(logits, labels, attention_mask):
    """ 
    args:
        logis: torch.Tensor, shape (batch_size, seq_len, vocab_size)
        labels: torch.Tensor, shape (batch_size, seq_len)
        attention_mask: torch.Tensor, shape (batch_size, seq_len) the original input text is masked with 0

    return:
        mean_log_probs: torch.Tensor, shape (batch_size,)
    """
    log_probs = torch.log_softmax(logits, dim=-1)
    # correct_log_probs = log_probs[range(log_probs.shape[0]), labels]

    # Flatten the tensors to simplify indexing
    batch_size, seq_len, vocab_size = logits.size()
    labels = labels.view(-1)  # flatten to (batch_size * seq_len)
    log_probs_flat = log_probs.view(-1, vocab_size)

    # Extract the log probabilities for the correct labels
    correct_log_probs_flat = log_probs_flat[range(batch_size * seq_len), labels]

    # Reshape to (batch_size, seq_len)
    correct_log_probs = correct_log_probs_flat.view(batch_size, seq_len)

    # Mask out the original input texts, only keep the generated tokens
    masked_log_probs = correct_log_probs * attention_mask

    # Calculate the mean log probability for each sequence
    mean_log_probs = masked_log_probs.sum(dim=-1)/ attention_mask.sum(dim=-1)

    return mean_log_probs, correct_log_probs


class MaskGeneratingModel(nn.Module):
    def __init__(self, hidden_size=4096, mlp_hidden_dim=1024, mlp_bottleneck_dim=768, mlp_num_blocks=2):
        """ 
        hidden_size: int
            The hidden size of the output of the generative model, 4096 for llama3
        """
        super().__init__()

        self.hidden_size = hidden_size

        self.explain_map = MLP(input_dim=hidden_size, 
                               hidden_dim=mlp_hidden_dim, 
                               output_dim=1, 
                               num_blocks=mlp_num_blocks, 
                               bottleneck_dim=mlp_bottleneck_dim) # takes [N, L, hidden_size] outputs [N, L, 1]
        self.logit_scale = nn.Parameter(torch.tensor(1.0))

        def bce_loss(input, target, context_mask):
            # input = input * context_mask
            # target = (target * context_mask).long()
            loss = - target * torch.log(input)  - (1 - target) * torch.log(1 - input)
            loss = (loss * context_mask).sum(-1) / context_mask.sum(dim=-1)
            return loss

        self.bce_loss = bce_loss
    
    def forward(self, pred_features):
        """ 
        pred_features: torch.Tensor of shape [N, L, hidden_size]
        """
        mask_logits = self.explain_map(pred_features).squeeze(-1) # [N, L]
        # mask_logits = F.normalize(mask_logits, p=2, dim=-1) # normalize the logits
        mask_logits = mask_logits * self.logit_scale.exp()
        return mask_logits # [batch_size, seq_len]
    
    @torch.no_grad()
    def generate_mask(self, mask_logits, context_mask):
        """
        generate mask based on a Bernoulli distribution with the probabilities of the given logits 
        args:
            mask_logits: torch.Tensor, shape (batch_size, seq_len)
            context_mask: torch.Tensor, shape (batch_size, seq_len), the context texts 
                (for example, the chatbot instruction template, not including the user inputs) are masked with 0.
                This mask strategy is to ensure we only focus on the real user inputs.
        return:
            mask: torch.Tensor, shape (batch_size, seq_len), with contexts being all 1s and user inputs being randomly masked.
        """
        # labels[:, :-1] = labels[:, 1:]
        # labels[:, -1] = pad_token_id
        mask_probs = torch.sigmoid(mask_logits) # (batch_size, seq_len)
        mask = torch.bernoulli(mask_probs) # (batch_size, seq_len)
        mask = mask.clone()
        mask[:, 1:] = mask[:, :-1] # shift the mask to the left
        mask[:, 0] = 0 # the last token should be masked
        mask = context_mask * mask + (1. - context_mask) # (batch_size, seq_len)
        return mask # this could be used as the attention mask for the generative model
    
    @torch.no_grad()
    def sample_one_batch(self, input_ids, attention_mask, mask_logits, context_mask):
        """ 
        args:
            input_ids: torch.Tensor, shape (batch_size, seq_len)
            attention_mask: torch.Tensor, shape (batch_size, seq_len), the attention_mask generated automatically by tokenizer, usually all 1s
            mask_logits: torch.Tensor, shape (batch_size, prompt_len), the logits for the mask generation
            context_mask: torch.Tensor, shape (batch_size, prompt_len), the context texts 
                (for example, the chatbot instruction template, not including the user inputs) are masked with 0.
        """
        #  get the mask of interest, i.e., for the user inputs, then pad to match the shape of other masks (of the full length)
        seq_len = input_ids.size(1)
        prompt_len = mask_logits.size(1)
        mask = self.generate_mask(mask_logits=mask_logits, context_mask=context_mask) # (batch_size, prompt_len)
        pad_length = seq_len - prompt_len
        padded_mask = F.pad(mask, (0, pad_length), mode='constant', value=1)

        # get the masked attention_mask
        masked_attention_mask = attention_mask * padded_mask

        # get the padded input_ids
        masked_input_ids = tokenizer.pad_token_id * torch.ones_like(input_ids) * (1 - masked_attention_mask).long() + input_ids * masked_attention_mask.long()

        return masked_input_ids, masked_attention_mask, mask
    
    def sample_one_batch_multi_samples(self, input_ids, attention_mask, mask_logits, context_mask, num_samples=5):
        """ 
        sample multiple samples for each input
        """
        batch_size, seq_len = input_ids.size()
        samples = []
        for _ in range(num_samples):
            masked_input_ids, masked_attention_mask, user_input_mask = self.sample_one_batch(input_ids, attention_mask, mask_logits, context_mask)
            samples.append((masked_input_ids, masked_attention_mask, user_input_mask))
        return samples
    
    @torch.no_grad()
    def calculate_sim(self, model, input_ids, attention_mask, response_mask, labels):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        logits = outputs.logits.float()
        # shift labels and response_mask to the left by one to match the next token prediction format
        labels = labels.clone()
        labels[:, :-1] = labels[:, 1:]
        labels[:, -1] = -100 # a random value that will be masked by the shift_response_mask
        shift_response_mask = response_mask.clone()
        shift_response_mask[:, :-1] = shift_response_mask[:, 1:]
        shift_response_mask[:, -1] = 0 # mast be zero.
        sim, correct_logprob = similarity_measure(logits, labels, shift_response_mask)
        return sim, correct_logprob
    
    @torch.no_grad()
    def get_reward(self, sim, sim_gt):
        reward = torch.exp(sim - sim_gt)
        return reward
    
    def loss_func(self, 
                  model, 
                  gen_tokens,
                  gen_attention_mask, 
                  context_mask,
                  mask_logits,
                  response_mask,
                  num_samples=5):
        # obtain the mask_prob
        mask_prob = torch.sigmoid(mask_logits)

        # obtain the perturbed_samples
        perturbed_samples = self.sample_one_batch_multi_samples(gen_tokens, gen_attention_mask, mask_logits, context_mask, num_samples=num_samples)

        # obtain the groundtruth similarity (similarity of the generated tokens without perturbation)
        reward_loss = 0
        sim_gt, _ = self.calculate_sim(model, gen_tokens, gen_attention_mask, response_mask, labels=gen_tokens)
        for perturbed_input_ids, perturbed_attention_mask, user_input_mask in perturbed_samples:
            # obtain the similarity of the perturbed samples
            sim, _ = self.calculate_sim(model, perturbed_input_ids, perturbed_attention_mask, response_mask, labels=gen_tokens)
            reward = self.get_reward(sim, sim_gt)
            reward_loss += (reward * self.bce_loss(mask_prob, user_input_mask, context_mask)).mean(dim=-1)
        reward_loss = reward_loss / num_samples
        reward_loss = reward_loss.mean()
        mask_loss = (mask_prob * context_mask / context_mask.sum(dim=-1, keepdim=True)).sum(dim=-1)
        mask_loss = mask_loss.mean()

        loss = reward_loss + 0.01 * mask_loss
        return {loss: loss, reward_loss: reward_loss, mask_loss: mask_loss}
    
    def compute_similarity(self, logits, labels, attention_mask):
        """ 
        compute the mean log_probs for the generated tokens
        """
        mean_log_probs = similarity_measure(logits, labels, attention_mask)
        return mean_log_probs
    



# Train model

In [5]:
mask_gen_model = MaskGeneratingModel(hidden_size=4096, mlp_hidden_dim=1024, mlp_bottleneck_dim=768, mlp_num_blocks=2)
mask_gen_model.to(device)


from tqdm import tqdm

# Set pad_token_id if it is not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

pad_token_id = tokenizer.pad_token_id

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

optimizer = torch.optim.Adam(mask_gen_model.parameters(), lr=1e-5)

In [6]:


for epoch in range(10):
    pbar = tqdm(train_dataloader)
    for idx, data in enumerate(pbar):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        context_mask = data['context_mask'].to(device)
        # get generated texts
        gen_outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
        gen_tokens = gen_outputs.sequences
        pad_length = gen_tokens.size(1) - input_ids.size(1)
        # get the attention mask for the generated tokens, and also mask the padding tokens
        gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
        # (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
        unpaded_token_mask = (gen_tokens != pad_token_id).long()
        unpaded_token_mask[:, :-pad_length] = 1
        gen_attention_mask = gen_attention_mask * unpaded_token_mask
        # get the response mask, which is the mask for the generated tokens (the user inputs are masked with 0)
        response_mask = gen_attention_mask.clone()
        response_mask[:, :-pad_length] = 0 # TODO: 有问题. 有问题吗？

        # Get the last hidden state for the prompt sequence
        with torch.no_grad():
            prompt_outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True, return_dict=True)
            last_hidden_state = prompt_outputs.hidden_states[-1]
            last_hidden_state = last_hidden_state.float()
        
        mask_logits = mask_gen_model(last_hidden_state)

        loss, reward_loss, mask_loss = mask_gen_model.loss_func(model, gen_tokens, gen_attention_mask, context_mask, mask_logits, response_mask, num_samples=5)
        
        pbar.set_description(f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss.item():.4f}, " 
                             f"Reward Loss = {reward_loss.item():.4f}, "
                            #  f"reward_mean = {reward_mean.item():.4f}, "
                            #  f"Regret Loss = {loss_dict['regret_loss'].item():.4f}, "
                             f"Mask = {mask_loss.item():.4f} "
                            #  f"alt_mask_loss = {loss_dict['alt_mask_loss'].item():.4f} "
                            #  f"mask_mean = {loss_dict['mask_mean'].item():.4f} "
                            #  f"prob_mean = {loss_dict['prob_mean'].item():.4f} "
                             )
        # Train the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #                      )
        if idx % 10 == 0:
            print()
        if idx == 200:
            torch.save(mask_gen_model.state_dict(), f'saved_model/mask_gen_model_{epoch}_{idx}.pth') 
            break
    break

            
        #     


Epoch 1, Step 1: Loss = 42.8870, Reward Loss = 42.8820, Mask = 0.5018 :   0%|          | 1/1563 [00:01<46:47,  1.80s/it]

Epoch 1, Step 5: Loss = 0.6485, Reward Loss = 0.6435, Mask = 0.4975 :   0%|          | 5/1563 [00:07<40:55,  1.58s/it]  


KeyboardInterrupt: 

# backup

In [ ]:


for epoch in range(10):
    pbar = tqdm(train_dataloader)
    for idx, data in enumerate(pbar):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        context_mask = data['context_mask'].to(device)
        # get generated texts
        gen_outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
        gen_tokens = gen_outputs.sequences
        pad_length = gen_tokens.size(1) - input_ids.size(1)
        # get the attention mask for the generated tokens, and also mask the padding tokens
        gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
        # (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
        unpaded_token_mask = (gen_tokens != pad_token_id).long()
        unpaded_token_mask[:, :-pad_length] = 1
        gen_attention_mask = gen_attention_mask * unpaded_token_mask
        # get the response mask, which is the mask for the generated tokens (the user inputs are masked with 0)
        response_mask = gen_attention_mask.clone()
        response_mask[:, :-pad_length] = 0 # TODO: 有问题. 有问题吗？
        # Get the last hidden state for the prompt sequence
        with torch.no_grad():
            prompt_outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True, return_dict=True)
            last_hidden_state = prompt_outputs.hidden_states[-1]
            last_hidden_state = last_hidden_state.float()
        
        mask_logits = mask_gen_model(last_hidden_state)

        
        # mask = mask_gen_model.generate_mask(mask_logits=mask_logits, context_mask=context_mask)
        perturbed_input_ids, masked_attention_mask, user_input_mask = mask_gen_model.sample_one_batch(input_ids=gen_tokens, 
                                                                attention_mask=gen_attention_mask, 
                                                                mask_logits=mask_logits, 
                                                                context_mask=context_mask)
        
        # Get the logits of the perturbed input concatenated with the response
        with torch.no_grad():
            outputs = model(input_ids=perturbed_input_ids, attention_mask=masked_attention_mask, return_dict=True)
            logits = outputs.logits.float()
        
        # Get the ground truth with the original input concatenated with the response
        with torch.no_grad():
            outputs_gt = model(input_ids=gen_tokens, attention_mask=gen_attention_mask, return_dict=True)
            logits_gt = outputs_gt.logits.float()
        
        # Compute the similarity between the logits and the response labels
        labels = gen_tokens.clone()
        labels[:, :-1] = labels[:, 1:]
        labels[:, -1] = pad_token_id
        shift_response_mask = response_mask.clone()
        shift_response_mask[:, :-1] = shift_response_mask[:, 1:]
        shift_response_mask[:, -1] = 0
        sim, correct_logprob = mask_gen_model.compute_similarity(logits, labels, shift_response_mask)
        sim_gt, correct_logprob_gt = mask_gen_model.compute_similarity(logits_gt, labels, shift_response_mask)

        # PPO
        
        reward = torch.exp(sim - sim_gt)
        mask_prob = torch.sigmoid(mask_logits)
        # bce_loss = nn.BCELoss(reduction='none')
        # reward_loss = reward.unsqueeze(-1) * bce_loss(context_mask * mask_prob, context_mask * user_input_mask) # consider manually implements the BCE loss
        def bce_loss(input, target, context_mask):
            # input = input * context_mask
            # target = (target * context_mask).long()
            loss = - target * torch.log(input)  - (1 - target) * torch.log(1 - input)
            loss = (loss * context_mask).sum(-1) / context_mask.sum(dim=-1)
            return loss
        reward_loss = reward * bce_loss(mask_prob, user_input_mask, context_mask)
        reward_loss = reward_loss.mean()
        # reduce the probability of the masked tokens, weighted by reward.
        # mask_loss = (mask_prob * context_mask * (1 - user_input_mask)).sum(dim=-1) / (context_mask * (1 - user_input_mask)).sum(dim=-1)
        mask_loss = (mask_prob * context_mask / context_mask.sum(dim=-1, keepdim=True)).sum(dim=-1)
        mean_mask = mask_loss.mean()

        # mask_loss = torch.relu(mask_loss + reward - 0.7)
        mask_loss = mask_loss.mean()
        reward_mean = reward.mean()

        loss = reward_loss + 0.01 * mask_loss
        pbar.set_description(f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss.item():.4f}, " 
                             f"Reward = {reward.mean().item():.4f},"
                             f"Reward Loss = {reward_loss.item():.4f}, "
                            #  f"reward_mean = {reward_mean.item():.4f}, "
                            #  f"Regret Loss = {loss_dict['regret_loss'].item():.4f}, "
                             f"Mask = {mean_mask.item():.4f} "
                            #  f"alt_mask_loss = {loss_dict['alt_mask_loss'].item():.4f} "
                            #  f"mask_mean = {loss_dict['mask_mean'].item():.4f} "
                            #  f"prob_mean = {loss_dict['prob_mean'].item():.4f} "
                             )
        # Train the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #                      )
        if idx % 10 == 0:
            print()
        if idx == 200:
            torch.save(mask_gen_model.state_dict(), f'saved_model/mask_gen_model_{epoch}_{idx}.pth') 
            break
    break

            
        #     


Epoch 1, Step 1: Loss = 0.6389, Reward = 0.9133,Reward Loss = 0.6341, Mask = 0.4811 :   0%|          | 1/1563 [00:01<27:25,  1.05s/it]

Epoch 1, Step 11: Loss = 0.6613, Reward = 0.9479,Reward Loss = 0.6566, Mask = 0.4727 :   1%|          | 11/1563 [00:11<27:48,  1.08s/it]

Epoch 1, Step 21: Loss = 0.5991, Reward = 0.8628,Reward Loss = 0.5944, Mask = 0.4736 :   1%|▏         | 21/1563 [00:23<28:26,  1.11s/it]

Epoch 1, Step 31: Loss = 0.5897, Reward = 0.8481,Reward Loss = 0.5850, Mask = 0.4664 :   2%|▏         | 31/1563 [00:34<28:00,  1.10s/it]

Epoch 1, Step 41: Loss = 0.5122, Reward = 0.7359,Reward Loss = 0.5075, Mask = 0.4660 :   3%|▎         | 41/1563 [00:46<30:48,  1.21s/it]

...

Epoch 1, Step 171: Loss = 0.6141, Reward = 0.8919,Reward Loss = 0.6097, Mask = 0.4328 :  11%|█         | 171/1563 [03:12<25:24,  1.09s/it]

Epoch 1, Step 181: Loss = 0.5319, Reward = 0.7701,Reward Loss = 0.5276, Mask = 0.4354 :  12%|█▏        | 181/1563 [03:24<27:54,  1.21s/it]

Epoch 1, Step 191: Loss = 0.6121, Reward = 0.8784,Reward Loss = 0.6077, Mask = 0.4415 :  12%|█▏        | 191/1563 [03:35<25:25,  1.11s/it]

Epoch 1, Step 201: Loss = 0.5076, Reward = 0.7319,Reward Loss = 0.5032, Mask = 0.4392 :  13%|█▎        | 200/1563 [03:46<24:50,  1.09s/it]

Epoch 1, Step 201: Loss = 0.5076, Reward = 0.7319,Reward Loss = 0.5032, Mask = 0.4392 :  13%|█▎        | 200/1563 [03:47<25:53,  1.14s/it]


In [ ]:
reward

tensor([0.2313, 0.6622, 0.9999, 0.8179, 1.0002, 0.9982, 0.9743, 0.0392, 0.6849,
        0.2606, 0.9711, 0.9964, 0.8578, 0.8377, 0.5780, 1.0005],
       device='cuda:0')

# Demo examples

In [ ]:
import numpy as np
idx = 0
from captum.attr import visualization as viz

# tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
# texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
# texts = "I don't like this movie."
texts = "I really love this film."
test_text = [{"text": texts, "label": None}]
test_inputs = collate_fn(test_text).to(device)

# test_inputs = next(iter(train_dataloader)).to(device)
tokens = tokenizer.convert_ids_to_tokens(test_inputs['input_ids'][0])

# generate the answer for the test inputs
gen_outputs = model.generate(
            input_ids=test_inputs['input_ids'],
            attention_mask=test_inputs['attention_mask'],
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
gen_tokens = gen_outputs.sequences
pad_length = gen_tokens.size(1) - input_ids.size(1)
# get the attention mask for the generated tokens, and also mask the padding tokens
gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
# (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
unpaded_token_mask = (gen_tokens != pad_token_id).long()
unpaded_token_mask[:, :-pad_length] = 1
gen_attention_mask = gen_attention_mask * unpaded_token_mask

with torch.no_grad():
    prompt_outputs = model(input_ids=test_inputs['input_ids'], attention_mask=test_inputs['attention_mask'], output_hidden_states=True, return_dict=True)
    last_hidden_state = prompt_outputs.hidden_states[-1].float()
    mask_logits = mask_gen_model(last_hidden_state)

# Function to clean tokens
def clean_token(token):
    # Remove special characters like "Ġ" or "Ċ"
    return token.replace("Ġ", "").replace("Ċ", "").replace("Ġ", "").replace("Ċ", "").replace("Ġ", "")

# Apply cleaning to each token
tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
tokens = [clean_token(token) for token in tokens]

def normalize_except_zeros(array):
    # Create a mask to identify non-zero elements
    mask = array != 0
    
    # Extract non-zero elements
    non_zero_elements = array[mask]
    
    # Normalize non-zero elements
    min_val = np.min(non_zero_elements)
    max_val = np.max(non_zero_elements)
    normalized_non_zero_elements = (non_zero_elements - min_val) / (max_val - min_val)
    
    # Create a copy of the original array to preserve zero values
    normalized_array = np.copy(array)
    
    # Assign normalized values back to the corresponding positions
    normalized_array[mask] = normalized_non_zero_elements
    
    return normalized_array

expl = (torch.sigmoid(mask_logits) * test_inputs['context_mask'])[idx]
expl = F.pad(expl, (0, len(tokens) - expl.size(0)), mode='constant', value=0)
expl_raw = expl.detach().cpu().numpy()
# expl = (expl - 0) / (expl_raw.max(axis=-1) - 0)


expl = normalize_except_zeros(expl_raw)


# expl = response_mask[idx]


vis_data_records = [viz.VisualizationDataRecord(
                                expl,
                                0,
                                0,
                                0,
                                0,
                                1,       
                                tokens,
                                1)]
                            
viz.visualize_text(vis_data_records)

In [ ]:
expl_raw

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.31899488, 0.25874132, 0.20597358,
       0.19858557, 0.19966002, 0.19903973, 0.20377652, 0.20697087,
       0.20099454, 0.20134583, 0.21013303, 0.20380026, 0.19403715,
       0.20613465, 0.19711944, 0.202277  , 0.19362119, 0.19564825,
       0.19992065, 0.19589779, 0.20217337, 0.1978755 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ], dtype=float32)

In [ ]:
mask_logits

tensor([[-0.0107, -1.2442, -1.2870, -1.3151, -0.9391, -1.2817, -0.7358, -0.9423,
         -0.9527, -1.0400, -1.2380, -1.0117, -1.4007, -1.5120, -1.4868, -1.4004,
         -1.3498, -1.3791, -1.3741, -1.4055, -1.3644, -1.4412, -1.2876, -1.3821,
         -1.5013, -1.4254, -1.1390, -1.3879, -1.0882, -1.3224, -0.9427, -1.5303,
         -1.5204, -1.0970, -1.0034, -1.0524, -1.1115, -1.3546, -1.4680, -1.3962,
         -1.3168, -1.3535, -1.6327, -1.6268, -1.6325, -1.5892, -1.5745, -1.5480,
         -1.6264, -1.6288, -1.1720, -1.1258, -1.0241, -1.0858, -0.8437]],
       device='cuda:0')

In [ ]:
expl = (torch.sigmoid(mask_logits) * test_inputs['context_mask'])[idx]
expl = F.pad(expl, (0, len(tokens) - expl.size(0)), mode='constant', value=0)
expl.detach().cpu().numpy()

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.34749058, 0.31645873, 0.292287  ,
       0.28547096, 0.30837953, 0.1781299 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ], dtype=float32)

In [ ]:
expl = expl.detach().cpu().numpy()
non_zero_elements = expl[expl!=0]
# Normalize non-zero elements
min_val = np.min(non_zero_elements)
max_val = np.max(non_zero_elements)
normalized_non_zero_elements = (non_zero_elements - min_val) / (max_val - min_val)
normalized_non_zero_elements

array([1.        , 0.8167707 , 0.67404723, 0.6338016 , 0.7690665 ,
       0.        ], dtype=float32)

In [ ]:
non_zero_elements

tensor([0.3475, 0.3165, 0.2923, 0.2855, 0.3084, 0.1781], device='cuda:0')

In [ ]:
expl = normalize_except_zeros(expl.detach().cpu().numpy())
expl

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.8167707 , 0.67404723,
       0.6338016 , 0.7690665 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ], dtype=float32)

In [ ]:
print(correct_logprob[0,:10])
print(correct_logprob_gt[0,:10])

tensor([-11.8911, -12.1255, -11.0005, -10.5004,  -9.8753,  -9.7502, -10.0002,
        -10.2501, -10.1251, -10.2501], device='cuda:0')
tensor([-7.7515, -7.1261, -7.5007, -7.1260, -6.3769, -5.8780, -6.2521, -6.2521,
        -6.0027, -6.1274], device='cuda:0')


In [ ]:
gen_tokens

tensor([[128009, 128009, 128009,  ...,   4632,     13, 128009],
        [128000, 128006,   9125,  ..., 128009, 128009, 128009],
        [128009, 128009, 128009,  ..., 128009, 128009, 128009],
        ...,
        [128009, 128009, 128009,  ..., 128009, 128009, 128009],
        [128009, 128009, 128009,  ..., 128009, 128009, 128009],
        [128009, 128009, 128009,  ..., 128009, 128009, 128009]],
       device='cuda:0')

# Demo examples 

In [ ]:
print(expl)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [ ]:

print(response_mask[idx])
# print("context_mask\n", context_mask[idx])
# print("user_input_mask\n",user_input_mask[idx].long())
print("masked_attention_mask\n",masked_attention_mask[idx].long())
# print("response_mask\n", response_mask[idx].long())
print("gen_attention_mask\n", gen_attention_mask[idx].long())


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
masked_attention_m

In [ ]:
tokenizer.decode(perturbed_input_ids[idx] * masked_attention_mask[idx].long())

'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a chatbot for sentimate analysis. You can help users with their questions via concise responses of POSITIVE, NEGATIVE OR NEUTURAL with a brief explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThis!!!!!!!!! any!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! by!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! bag!!!!!!! the!!!!!!!!!!!!!!!!!!<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nNEGATIVE\n\nThe reviewer expresses their disappointment and frustration with the movie, calling it a "classic" only because of its sentiment rather than its quality. They criticize the plot as "unbelievable", "unrealistic", and "ludicrous", and highlight specific scenes as unrealistic, such as a regiment marching into an ambush. The tone is overwhelmingly negative.!!!!!!!!!!!!!!!!!!!!!'

In [ ]:
torch.sigmoid(mask_logits)[idx]

tensor([7.3034e-02, 2.3672e-01, 1.6484e-01, 5.7779e-02, 4.5203e-02, 3.7355e-02,
        5.0271e-02, 6.9986e-02, 6.1506e-02, 4.2663e-02, 4.4438e-02, 7.5177e-02,
        3.5435e-02, 5.5367e-02, 3.7714e-02, 3.1168e-02, 4.1736e-02, 4.3801e-02,
        3.1036e-02, 3.4357e-02, 5.7400e-02, 3.6266e-02, 4.7387e-02, 9.0895e-02,
        4.7148e-02, 5.1576e-02, 5.1955e-02, 4.8488e-02, 6.0373e-02, 6.3567e-02,
        6.8870e-02, 8.2406e-02, 6.4658e-02, 5.7909e-02, 6.6989e-02, 5.4006e-02,
        5.4770e-02, 1.3873e-01, 7.5017e-01, 1.8232e-01, 2.9083e-02, 7.7919e-02,
        2.8039e-01, 1.2871e-01, 6.7355e-01, 5.4293e-01, 5.2408e-02, 1.3911e-01,
        7.0787e-01, 3.0805e-01, 3.1633e-01, 1.9815e-02, 7.9728e-02, 1.2592e-01,
        2.8685e-01, 7.6748e-01, 3.5774e-01, 7.0721e-01, 8.3341e-01, 1.0890e-01,
        4.6710e-01, 4.1535e-02, 1.5163e-02, 9.8794e-02, 1.5306e-01, 4.1573e-02,
        3.1574e-01, 7.2187e-01, 3.3909e-02, 1.4620e-01, 2.2021e-01, 2.9110e-01,
        2.8944e-02, 2.2404e-01, 3.1191e-

In [ ]:
from captum.attr import visualization as viz

tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
# Function to clean tokens
def clean_token(token):
    # Remove special characters like "Ġ" or "Ċ"
    return token.replace("Ġ", "").replace("Ċ", "").replace("Ġ", "").replace("Ċ", "").replace("Ġ", "")

# Apply cleaning to each token
tokens = [clean_token(token) for token in tokens]

expl = (torch.sigmoid(mask_logits) * context_mask)[idx]
expl = F.pad(expl, (0, len(tokens) - expl.size(0)), mode='constant', value=0)


vis_data_records = [viz.VisualizationDataRecord(
                                expl,
                                0,
                                0,
                                0,
                                0,
                                1,       
                                tokens,
                                1)]
                            
viz.visualize_text(vis_data_records)

AttributeError: 'LlamaForCausalLM' object has no attribute 'tokenizer'